In [1]:
%matplotlib inline
import nibabel as nib
import numpy as np
from scipy import optimize
import matplotlib.pyplot as plt
import matplotlib.image as mpimg


In [2]:
import sys
# Specify the location of this repository on your pc:
path_to_chi_to_fm_fft = "C:/Users/User/msc_project/susceptibility-to-fieldmap-fft"
sys.path.append(path_to_chi_to_fm_fft)
#sys.path.append('/Users/evaalonsoortiz/Documents/python/susceptibility-to-fieldmap-fft/')
from functions import compute_fieldmap

In [17]:
def f(x):

    # load simulated susceptibility map in ppm
    chi_img = nib.load(path_chimap)
    chi_data = chi_img.get_fdata()
    
    # load segmentation labels map
    ROIs_img = nib.load(path_labels)
    ROIs_data = ROIs_img.get_fdata()
    # find indicies that have label = 256, these voxels should correspond to the spinal cord
    #ind_sc = np.where(ROIs_data == 256) # For ISMRM chimap, we are not segmenting SC now
    # find indicies that have label = 7 or 8, these voxels should correspond to the trachea (t) and lungs (l); ind_tl
    ind_tl = np.where((ROIs_data == 7) or (ROIs_data == 8))
    
    # load simulated field map in Hz
    b0meas_img = nib.load("fm_Hz.nii.gz")
    b0meas_data = b0meas_img.get_fdata()

    # find indicies that have chi = -4.2, these voxels should correspond to the trachea (t) and lungs (l); ind_tl
    #ind_tl = np.where(chi_data == -4.2)

    # assign variable "x" to indicies corresponding to trachea/lungs
    chi_data[ind_tl] = x

    # compute the field distribution in ppm for the chi_data distribution using the compute_bz function from the susceptibility-to-fieldmap-fft package
    z =  compute_fieldmap.compute_bz(chi_data)
    # convert field distribution from ppm to Hz (assuming 3T)
    z = z*3*42.58 # deltaB0 [Hz] = deltaB0 [ppm] * B0 [T] * 1e-6 * 42.58*e6 [Hz/T]

    return (np.linalg.norm(b0meas_data[ind_sc] - z[ind_sc]))

In [18]:
minimum = min_chi_tl = optimize.fmin(f, -4.2)
minimum[0]

ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()